# Setup

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2 as cv
from PIL import Image
import re
import glob
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf
import math

plt.ion()   # interactive mode

BOTAR AQUI SUA PASTA DO NOTEBOOK

In [2]:
# pasta_do_notebook = '/content/drive/MyDrive/Blur_CLas'
pasta_do_notebook = '.'
os.chdir(pasta_do_notebook)

In [3]:
os.getcwd()

'd:\\daniel_moreira\\reconhecimento_de_padroes\\reconhecimento_de_padroes\\blurdetect'

# Dataset

In [4]:
image_path = []

base_path = 'blur_dataset'
for caminho in os.listdir(base_path):
  pasta = os.path.join(base_path, caminho)
  for image in os.listdir(pasta):
    image_path = os.path.join(pasta, image)

In [5]:
defocused_path = 'blur_dataset/defocused_blurred/*'
motion_path = 'blur_dataset/motion_blurred/*'
sharp_path = 'blur_dataset/sharp/*'

In [6]:
ids = []

for caminho in glob.glob(sharp_path):
  partes = caminho.split('_')
  partes = partes[:-1]
  res = "_".join(partes)
  res = os.path.basename(res)
  ids.append(res)

ids.sort()

In [7]:
print(ids[:5])
print(ids[-5:])

['0_IPHONE-SE', '100_NIKON-D3400-35MM', '101_NIKON-D3400-35MM', '102_NIKON-D3400-35MM', '103_HUAWEI-P20']
['96_HONOR-7X', '97_HONOR-7X', '98_HONOR-7X', '99_HONOR-7X', '9_HUAWEI-P20']


In [8]:
split = int(0.8 * len(ids))
id_train, id_test = ids[:split], ids[split:]
split = int(0.5 * len(id_test))
id_test, id_val = id_test[:split], id_test[split:]

X_train, y_train = [], []
X_test, y_test = [], []
X_val, y_val = [], []

for caminho in glob.glob(defocused_path):
    if os.path.basename(caminho).split('.')[0][:-2] in id_train:
        X_train.append(caminho)
        y_train.append(1)
    elif os.path.basename(caminho).split('.')[0][:-2] in id_test:
        X_test.append(caminho)
        y_test.append(1)
    elif os.path.basename(caminho).split('.')[0][:-2] in id_val:
        X_val.append(caminho)
        y_val.append(1)

for caminho in glob.glob(motion_path):
    if os.path.basename(caminho).split('.')[0][:-2] in id_train:
        X_train.append(caminho)
        y_train.append(1)
    elif os.path.basename(caminho).split('.')[0][:-2] in id_test:
        X_test.append(caminho)
        y_test.append(1)
    elif os.path.basename(caminho).split('.')[0][:-2] in id_val:
        X_val.append(caminho)
        y_val.append(1)

for caminho in glob.glob(sharp_path):
    if os.path.basename(caminho).split('.')[0][:-2] in id_train:
        X_train.append(caminho)
        y_train.append(0)
    elif os.path.basename(caminho).split('.')[0][:-2] in id_test:
        X_test.append(caminho)
        y_test.append(0)
    elif os.path.basename(caminho).split('.')[0][:-2] in id_val:
        X_val.append(caminho)
        y_val.append(0)

In [9]:
print(len(X_train))
print(len(X_test))
print(len(X_val))
print(X_val[:5])
print(len(y_val))
print(y_val[:5])

831
105
105
['blur_dataset/defocused_blurred\\69_SAMSUNG-GALAXY-J3_F.jpg', 'blur_dataset/defocused_blurred\\6_HUAWEI-MATE20_F.jpg', 'blur_dataset/defocused_blurred\\70_HONOR-8X_F.jpg', 'blur_dataset/defocused_blurred\\71_SAMSUNG-GALAXY-A6_F.jpg', 'blur_dataset/defocused_blurred\\72_IPHONE-8_F.jpg']
105
[1, 1, 1, 1, 1]


In [10]:
# Function to create patches from an image with a specified patch size
def split_image(image, patch_size=256, qtd=1):

    height, width, _ = image.shape

    # Calculate the number of rows and columns needed for patches
    num_rows = math.ceil(height / patch_size)
    num_cols = math.ceil(width / patch_size)

    out_height = patch_size * num_rows
    out_width = patch_size * num_cols

    # Calculate the amount of padding needed
    pad_height = out_height - height
    pad_width = out_width - width

    # Pad the image with zeros if necessary
    if pad_height > 0 or pad_width > 0:
        image = cv.copyMakeBorder(image, 0, pad_height, 0, pad_width, cv.BORDER_CONSTANT, value=0)

    # Initialize an empty list to store the patches
    patches = []

    # Iterate through the image and extract patches
    for y in range(0, out_height, patch_size):
        for x in range(0, out_width, patch_size):
            patch = image[y:y+patch_size, x:x+patch_size]
            patches.append(patch)

    if qtd < 1 and qtd > 0:
        np.random.shuffle(patches)
        patches = patches[:int(len(patches)*qtd)]

    return np.array(patches)

In [11]:
ex_image = cv.imread(X_train[-1])
ex_patches = split_image(ex_image, 256, 0.2)
print(ex_image.shape)
print(ex_patches.shape)

(5120, 3840, 3)
(60, 256, 256, 3)


In [12]:
# import numpy as np
# from PIL import Image

# # Define your NumPy array of images
# image_array = ex_patches

# # Loop through each image in the array
# for i in range(len(image_array)):
#     # Convert the NumPy array to a PIL image
#     image = Image.fromarray(image_array[i])

#     # Display the image in the Jupyter notebook
#     display(image)

In [13]:
def patchify(X, y, qtd_patches=1):
    X_patches, y_patches = [], []
    for i in range(len(X)):
        patches = split_image(cv.imread(X[i]), 256, qtd_patches)
        X_patches.extend(patches)
        for patch in patches:
            if y[i] == 1:
                y_patches.append(1)
            else:
                y_patches.append(0)
    return np.array(X_patches), np.array(y_patches)

In [ ]:
X_train, y_train = patchify(X_train, y_train, 0.01)
X_test, y_test = patchify(X_test, y_test, 0.01)
X_val, y_val = patchify(X_val, y_val, 0.01)

In [15]:
if not os.path.exists('datasets'):
    os.makedirs('datasets')

np.save('datasets/X_train.npy', X_train)
np.save('datasets/y_train.npy', y_train)

np.save('datasets/X_test.npy', X_test)
np.save('datasets/y_test.npy', y_test)

np.save('datasets/X_val.npy', X_val)
np.save('datasets/y_val.npy', y_val)

In [16]:
print(len(X_train))
print(len(X_test))
print(len(X_val))

1482
213
204


# Treino

In [ ]:
X_train_patches = np.load('datasets/X_train.npy')
y_train_patches = np.load('datasets/y_train.npy')

X_val_patches = np.load('datasets/X_val.npy')
y_val_patches = np.load('datasets/y_val.npy')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

if not os.path.exists('weights'):
    os.mkdir('weights')

checkpoint = ModelCheckpoint(
    filepath='weightscheckpoint_path.h5',
    monitor='val_loss',  # Métrica para monitorar (pode ser 'val_loss' para perda de validação)
    save_best_only=True,  # Salva apenas o melhor modelo
    mode='min',  # 'max' se a métrica monitorada deve ser maximizada, 'min' se deve ser minimizada
    verbose=1  # Exibe mensagens durante o salvamento do modelo
)

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model

model = tf.keras.applications.densenet.DenseNet121(
    include_top=False,
    weights='imagenet',
    input_shape=(256, 256, 3)
)

top = GlobalAveragePooling2D()(model.output)
top = Dense(1024, activation='relu')(top)
top = Dense(1, activation='sigmoid')(top)
model = Model(inputs=model.input, outputs=top)

29084464/29084464 [==============================] - 0s 0us/step


In [ ]:
from tensorflow.keras.applications.densenet import preprocess_input
def preprocess_data(X, y, shuffle=False):
    # Shuffle the data if specified
    X = X/255
    if shuffle:
        indices = np.arange(len(X))
        np.random.shuffle(indices)
        X = X[indices]
        y = y[indices]

    return X, y

In [ ]:
X_train, y_train = preprocess_data(X_train, y_train)
X_test, y_test = preprocess_data(X_test, y_test)
X_val, y_val = preprocess_data(X_val, y_val)
print(X_train.shape, X_test.shape, X_val.shape)

(1482, 256, 256, 3) (213, 256, 256, 3) (204, 256, 256, 3)


In [ ]:
# Convert NumPy arrays to TensorFlow tensors
X_train = tf.convert_to_tensor(X_train)
y_train = tf.convert_to_tensor(y_train)

# Create a tf.data.Dataset
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

# Batch, shuffle, and repeat the dataset
batch_size = 16
dataset = dataset.shuffle(buffer_size=len(X_train))
dataset = dataset.batch(batch_size)
# dataset = dataset.repeat()

In [ ]:
# Compile o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treine o modelo
# r = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=50, callbacks=[checkpoint], batch_size = 8)
r = model.fit(dataset, validation_data=(X_val,y_val), epochs=50, callbacks=[checkpoint])

# Avalie o modelo no conjunto de teste
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Exiba as métricas calculadas
print("Loss:", test_loss)
print("Accuracy:", test_accuracy)
print("Validation", r.history['val_loss'])

Epoch 1/50
93/93 [==============================] - ETA: 0s - loss: 0.6717 - accuracy: 0.7274
Epoch 1: val_loss improved from inf to 0.68267, saving model to checkpoint_path.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


93/93 [==============================] - 109s 386ms/step - loss: 0.6717 - accuracy: 0.7274 - val_loss: 0.6827 - val_accuracy: 0.7059
Epoch 2/50
93/93 [==============================] - ETA: 0s - loss: 0.5322 - accuracy: 0.7713
Epoch 2: val_loss did not improve from 0.68267
93/93 [==============================] - 23s 243ms/step - loss: 0.5322 - accuracy: 0.7713 - val_loss: 0.6997 - val_accuracy: 0.7255
Epoch 3/50
93/93 [==============================] - ETA: 0s - loss: 0.5241 - accuracy: 0.7632
Epoch 3: val_loss did not improve from 0.68267
93/93 [==============================] - 23s 245ms/step - loss: 0.5241 - accuracy: 0.7632 - val_loss: 2.0463 - val_accuracy: 0.6176
Epoch 4/50
93/93 [==============================] - ETA: 0s - loss: 0.4708 - accuracy: 0.7989
Epoch 4: val_loss improved from 0.68267 to 0.54985, saving model to checkpoint_path.h5
93/93 [==============================] - 26s 284ms/step - loss: 0.4708 - accuracy: 0.7989 - val_loss: 0.5499 - val_accuracy: 0.7696
Epoch 5/

In [ ]:
loss = r.history['loss']
val_loss = r.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc1 = r.history['accuracy']
val_acc1 = r.history['val_accuracy']
plt.plot(epochs, acc1, 'y', label='Training acurácia')
plt.plot(epochs, val_acc1, 'r', label='Validation acurácia')
plt.title('Training and validation acurácia')
plt.xlabel('Epochs')
plt.ylabel('acurácia')
plt.legend()
plt.show()

# Teste

In [29]:
X_test = np.load('datasets/X_test.npy')
y_test = np.load('datasets/y_test.npy')

In [30]:
print(len(X_test))

213


In [31]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model

model = tf.keras.applications.densenet.DenseNet121(
    include_top=False,
    weights=None,
    input_shape=(256, 256, 3)
)

top = GlobalAveragePooling2D()(model.output)
top = Dense(1024, activation='relu')(top)
top = Dense(1, activation='sigmoid')(top)
model = Model(inputs=model.input, outputs=top)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

from tensorflow.keras.applications.densenet import preprocess_input
def preprocess_data(X, y, shuffle=False):
    # Shuffle the data if specified
    X = X/255
    if shuffle:
        indices = np.arange(len(X))
        np.random.shuffle(indices)
        X = X[indices]
        y = y[indices]

    return X, y

In [32]:
X_test, y_test = preprocess_data(X_test, y_test)

In [33]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model.load_weights(os.path.join(pasta_do_notebook, 'weights', 'checkpoint_path.h5'))

y_pred = model.predict(X_test)

y_pred_classes = np.round(y_pred)

accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes)
recall = recall_score(y_test, y_pred_classes)
f1 = f1_score(y_test, y_pred_classes)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.6667
Precision: 0.6667
Recall: 1.0000
F1 Score: 0.8000
